In [12]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.fillna(" ")
test = test.fillna(" ")
X_train = train['subject'] + ' ' + train['email']
y_train = train['spam']
X_test = test['subject'] + ' ' + test['email']

# for t in X_train[:10]:
#     print(t)

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("./bert_hugginggace")
# distilbert-base-uncased-finetuned-sst-2-english

pretrain_model = AutoModelForSequenceClassification.from_pretrained("./bert_hugginggace")

In [9]:
PAD, CLS = '[PAD]', '[CLS]'
max_seq_len = 400

def load_dataset(texts, labels):
    contents = []
    for t, label in zip(texts, labels):
        token = tokenizer.tokenize(t)
        token = [CLS] + token
        seq_len = len(token)
        mask = []
        token_ids = tokenizer.convert_tokens_to_ids(token)
        if len(token) < max_seq_len:
            mask = [1]*len(token) + [0]*(max_seq_len-len(token))
            token_ids = token_ids + [0]*(max_seq_len-len(token))
        else:
            mask = [1]*max_seq_len
            token_ids = token_ids[:max_seq_len]
            seq_len = max_seq_len
        contents.append((token_ids, int(label), seq_len, mask))
    return contents

load_dataset(X_train, y_train)

KeyboardInterrupt: 

In [11]:
def build_iter(datasets, batch_size, device):
    iter = datasetIter(datasets,batch_size,device)
    return iter
class datasetIter():
    def __init__(self,datasets,batch_size,device):
        self.datasets = datasets
        self.idx = 0
        self.device = device
        self.batch_size = batch_size
        self.batches = len(datasets)//batch_size
        self.residues = False
        if len(datasets)%batch_size != 0:
            self.residues = True
    def __next__(self):
        if self.residues and self.idx==self.batches:
            batch_data = self.datasets[self.idx * self.batch_size : len(self.datasets)]
            self.idx += 1
            batch_data = self._to_tensor(batch_data)
            return batch_data
        elif self.idx > self.batches:
            self.idx = 0
            raise StopIteration
        else:
            batch_data = self.datasets[self.idx * self.batch_size : (self.idx+1) * self.batch_size]
            self.idx += 1
            batch_data = self._to_tensor(batch_data)
            return batch_data
    def _to_tensor(self, datasets):
        x = torch.LongTensor([item[0] for item in datasets]).to(self.device)
        y = torch.LongTensor([item[1] for item in datasets]).to(self.device)
        seq_len = torch.LongTensor([item[2] for item in datasets]).to(self.device)
        mask = torch.LongTensor([item[3] for item in datasets]).to(self.device)
        return (x, seq_len, mask), y
    def __iter__(self):
        return self
    def __len__(self):
        if self.residues:
            return self.batches + 1
        else:
            return self.batches

In [14]:
bert_hidden = 768
num_classes = 2
class myModel(nn.Module):
    def __init__(self):
        super(myModel, self).__init__()
        self.pretrain_model = pretrain_model
        for param in self.pretrain_model.parameters():
            param.requires_grad = True
        self.fc = nn.Linear(bert_hidden, num_classes)
    def forward(self, x):
        context = x[0]
        mask = x[2]
        _, pooled = self.pretrain_model(context, attention_mask=mask,output_all_encoded_layers=False)
        out = self.fc(pooled)
        return out

In [ ]:
import torch.nn.functional as F
from sklearn import metrics
from transformers.optimization import AdamW

def train(model, )